# Model

In [1]:
!mkdir issue_1927

mkdir: cannot create directory ‘issue_1927’: File exists


In [2]:
%%writefile issue_1927/env

MODEL_NAME=Model
API_TYPE=REST
SERVICE_TYPE=MODEL
PERSISTENCE=0
SELDON_LOG_LEVEL=DEBUG

Overwriting issue_1927/env


In [105]:
%%writefile issue_1927/Model.py

import logging
import json
import time
import os


name = os.environ.get("SELDON_NAME", "unknown")


class Model:

    def predict(self, features, names=[], meta={}):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        time.sleep(.1)
        return features.tolist()
        
    def transform_input(self, features, names=[], meta=[]):
        return self.predict(features, names, meta)

    def transform_output(self, features, names=[], meta=[]):
        return self.predict(features, names, meta)

    def aggregate(self, features, names=[], meta=[]):
        logging.info(f"model features: {features}")
        logging.info(f"model names: {names}")
        logging.info(f"model meta: {meta}")
        time.sleep(.1)
        return [x.tolist() for x in features]
    
    def tags(self):
        return {f"{name}-timestamp": time.time()}

Overwriting issue_1927/Model.py


In [106]:
%%bash
cd issue_1927
s2i build -E env . seldonio/seldon-core-s2i-python37:1.1.1-rc rafalseldon/issue_1927:0.1
kind load docker-image rafalseldon/issue_1927:0.1 --name seldon

---> Installing application source...
Created wheel for PTable: filename=PTable-0.9.2-cp37-none-any.whl size=22906 sha256=2157b5329604f9bc97f6753b4d970a0f28dccee748ca6e146b1baa6788f25913
Stored in directory: /root/.cache/pip/wheels/22/cc/2e/55980bfe86393df3e9896146a01f6802978d09d7ebcba5ea56
Successfully built PTable
created path: ./licenses/license_info.csv
created path: ./licenses/license.txt
Build completed successfully
Image: "rafalseldon/issue_1927:0.1" with ID "sha256:c04068cd6f3003236480eb63bc91da8fafb53d9bf69a8f74e7b1ece52a65db6d" not present on node "seldon-control-plane"


# Deployment

In [107]:
%%writefile issue_1927/deploy.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: issue-1927
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
            
        - image: rafalseldon/issue_1927:0.1
          name: response-generator
          env:
          - name: SELDON_NAME
            value: response-generator  
                
        - image: rafalseldon/issue_1927:0.1
          name: model-a
          env:
          - name: SELDON_NAME
            value: model-a
                
        - image: rafalseldon/issue_1927:0.1
          name: to-tensor-output-transformer
          env:
          - name: SELDON_NAME
            value: to-tensor-output-transformer
                
        - image: rafalseldon/issue_1927:0.1
          name: to-tensor-transformer
          env:
          - name: SELDON_NAME
            value: to-tensor-transformer
                
        - image: rafalseldon/issue_1927:0.1
          name: image-crop
          env:
          - name: SELDON_NAME
            value: image-crop
                
        - image: rafalseldon/issue_1927:0.1
          name: downloader-1
          env:
          - name: SELDON_NAME
            value: downloader-1
                
        - image: rafalseldon/issue_1927:0.1
          name: downloader-2
          env:
          - name: SELDON_NAME
            value: downloader-2          
                
        - image: rafalseldon/issue_1927:0.1
          name: model-2
          env:
          - name: SELDON_NAME
            value: model-2
    graph:
        name: response-generator
        type: OUTPUT_TRANSFORMER
        children:
        - name: model-a
          type: OUTPUT_TRANSFORMER
          children:
          - name: to-tensor-output-transformer
            type: OUTPUT_TRANSFORMER
            children:
            - name: image-crop
              type: COMBINER
              children:
              - name: downloader-1
                type: TRANSFORMER
              - name: downloader-2
                type: TRANSFORMER
                children:
                - name: to-tensor-transformer
                  type: TRANSFORMER
                  children:
                  - name: model-2
                    type: MODEL
    name: example
    replicas: 1

Overwriting issue_1927/deploy.yaml


In [108]:
!kubectl delete -f issue_1927/deploy.yaml

seldondeployment.machinelearning.seldon.io "issue-1927" deleted


In [109]:
!kubectl apply -f issue_1927/deploy.yaml

seldondeployment.machinelearning.seldon.io/issue-1927 created


# Test

In [114]:
%%bash

curl -s -m 20 -H 'Content-Type: application/json' \
    -d '{"meta": {"tags": {"counter": 0}}, "data": {"names": ["input"], "ndarray": ["data"]}}' \
    http://localhost:8003/seldon/seldon/issue-1927/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0"
    ],
    "ndarray": [
      [
        "data"
      ],
      [
        "data"
      ]
    ]
  },
  "meta": {
    "tags": {
      "counter": 0,
      "downloader-1-timestamp": 1592229887.3287637,
      "downloader-2-timestamp": 1592229887.3278418,
      "image-crop-timestamp": 1592229887.6459646,
      "model-2-timestamp": 1592229887.54179,
      "model-a-timestamp": 1592229887.8638797,
      "response-generator-timestamp": 1592229887.973219,
      "to-tensor-output-transformer-timestamp": 1592229887.7547598,
      "to-tensor-transformer-timestamp": 1592229887.437887
    }
  }
}


In [115]:
import requests


r = requests.post(
    "http://localhost:8003/seldon/seldon/issue-1927/api/v1.0/predictions",
    json={"meta": {"tags": {"counter": 0}}, "data": {"names": ["input"], "ndarray": ["data"]}}
)


j = r.json()

In [116]:
data = {k: v for k, v in sorted(j["meta"]["tags"].items(), key=lambda item: item[1]) if "timestamp" in k}
m = min(data.values())
data = {k: round(10*(v-m), 1) for k, v in data.items()}

In [117]:
data

{'downloader-1-timestamp': 0.0,
 'downloader-2-timestamp': 0.0,
 'to-tensor-transformer-timestamp': 1.1,
 'model-2-timestamp': 2.2,
 'image-crop-timestamp': 3.2,
 'to-tensor-output-transformer-timestamp': 4.3,
 'model-a-timestamp': 5.4,
 'response-generator-timestamp': 6.5}